In [79]:
import pandas as pd
import numpy as np
import heapq


np.random.seed(seed=7)
# Load data
interactions = pd.read_csv('./data/interactions.csv', sep='\t')
items = pd.read_csv('./data_modified/item_profile.csv', sep='\t')
users = pd.read_csv('./data/user_profile.csv', sep='\t')
interactions

,user_id,item_id,interaction_type,created_at
0,2690450,1180447,1,1444806365
1,2690450,1981683,1,1444894156
2,732722,1266442,1,1446745768
3,1731019,265291,1,1445613604
4,1731019,2397541,1,1446822664
5,1731019,1860035,1,1445613601
6,1731019,2397541,3,1446822698
7,523641,926150,1,1446542202
8,884155,205039,1,1445330555
9,1053084,1617810,1,1442408546


In [74]:
# TODO: Precalculate all user-item scores.
def get_interactions_score(user_id, item_id):
    user_items = interactions[(interactions['user_id'] == user_id) & (interactions['item_id'] == item_id)]
    score = 0
    for i, row in user_items.iterrows():
        score += row['interaction_type']
    return score

def get_role_title_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    
    job_roles = user['jobroles'].split(',')
    job_titles = item['title'].split(',')
    score = 0
    for role in job_roles:
        for title in job_titles:
            if title == role:
                score += 1
                
    return score

def get_career_level_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['career_level'] == item['career_level']


def get_discipline_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['discipline_id'] == item['discipline_id']

def get_industry_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['industry_id'] == item['industry_id']

def get_regions_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['region'] == item['region']

W1 = 100
W2 = 10
W3 = 12
W4 = 10
W5 = 5
W6 = 2

def heuristic_score_for_item(user_id, item_id):
    return np.sum(np.asarray([W1, W2, W3, W4, W5, W6]) * np.asarray([
        get_interactions_score(user_id, item_id),
        get_role_title_match(user_id, item_id),
        get_career_level_match(user_id, item_id),
        get_discipline_match(user_id, item_id),
        get_industry_match(user_id, item_id),
        get_regions_match(user_id, item_id)
    ]))

# Works very slow, should precalculate scores.
def heuristic_score(user_id, rec_count=5):
    recommendations = []
    for _, row in items.iterrows():
        score = heuristic_score_for_item(user_id, row['id'])
        heapq.heappush(recommendations, score)
        
    top_scores = []
    for i in range(rec_count):
        top_scores.append(heapq.heappop(recommendations))
    return top_scores


# user = users[users['user_id'] == 1731019].iloc[0]
# item = items[items['id'] == 2397541].iloc[0]
# user_items = interactions[(interactions['interaction_type'] == 3)]

# print(user_items)
# print(item)
# print(user)

# print(heuristic_score_for_item(1731019, 2397541))

# print(heuristic_score(1286))


    

    

In [75]:
from sklearn.metrics.pairwise import cosine_similarity

id_to_idx = {}
idx_to_item = {}

for i, item in items.iterrows():
        id_to_idx[item['id']] = i
        idx_to_item[i] = item

# Collaborative filtering
def item_item_CF(items):
    return cosine_similarity(items, items)







    
    
        
    
    

In [89]:
from scipy.cluster.vq import kmeans2
import numpy as np
import math

# Clusterizition
def get_clusters(items_matrix, num_clusters=10):    
    return kmeans2(items_matrix, num_clusters)
    

# print(item_matrix[math.isnan(item_matrix)])
    
item_matrix = items.values
item_matrix[item_matrix == ''] = '0'
item_matrix = np.delete(items.values, [0, 1, 2, 5, 10], 1).astype(float)
centroid, item_clusters = get_clusters(item_matrix)
# print('item clusts', item_clusters)
similarities = {}

def get_item_cluster(item_id):
    idx = id_to_idx[item_id]
    return item_clusters[idx]

def get_cluster_items(cluster_id):
    return [item_idx for item_idx, cluster in enumerate(item_clusters) if cluster_id == cluster]

print(item_clusters[:100])
print('-------', get_cluster_items(5))

def recommend(user_id, rec_num=30):
    user_interactions = interactions[interactions['user_id'] == user_id]
    recommendations = []
    for _, item in user_interactions.iterrows():
#         print(item['item_id'])
        cluster_id = get_item_cluster(item['item_id'])
        cluster_items = get_cluster_items(cluster_id)[:100]
        if cluster_id not in similarities:
#             print(cluster_items)
            similarities[cluster_id] = item_item_CF(cluster_items)
        similarity = similarities[cluster_id]
        x = simalarity[id_to_idx[item['id']]]
        x = [(score, i) for i, score in enumerate(x)]
        x.sort()
        recommnedations.extend(x[:rec_num])
        
    recommendations.sort()
    return recommendations[:rec_num]

print(recommend(1731019))
        

c:\python3.6x64\lib\site-packages\scipy\cluster\vq.py:523: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


[6 2 2 6 6 2 6 6 2 6 6 2 2 2 2 2 6 6 2 6 2 2 2 6 2 2 2 6 6 2 6 2 6 6 2 2 6
 2 2 6 2 2 2 6 2 2 6 2 6 2 2 6 2 2 2 2 6 6 2 2 6 6 2 2 6 2 2 2 2 2 2 6 2 6
 2 6 2 2 2 2 2 2 6 2 2 2 6 2 2 6 2 2 2 6 6 2 6 6 6 2]
------- []


ValueError: Expected 2D array, got 1D array instead:
array=[   0.    3.    4.    6.    7.    9.   10.   16.   17.   19.   23.   27.
   28.   30.   32.   33.   36.   39.   43.   46.   48.   51.   56.   57.
   60.   61.   64.   71.   73.   75.   82.   86.   89.   93.   94.   96.
   97.   98.  105.  109.  113.  123.  128.  130.  131.  132.  133.  141.
  145.  147.  150.  158.  163.  166.  169.  172.  174.  181.  182.  183.
  186.  191.  194.  195.  201.  205.  209.  212.  213.  217.  218.  222.
  225.  228.  229.  232.  237.  238.  239.  240.  243.  244.  248.  250.
  268.  270.  272.  273.  279.  280.  282.  284.  286.  292.  294.  305.
  319.  320.  321.  324.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.